In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
demo = pd.read_csv('dataset/demo.csv')
demo.rename({
    'c0':'gender',
    'c1':'ocp'}, axis=1, inplace=True)

txn = pd.read_csv('dataset/txn.csv')
txn.rename({
    'n3':'date_idx'
}, axis=1, inplace=True)
txn.drop('t0', axis=1, inplace=True)

train_set = pd.read_csv('dataset/train.csv')
test_set = pd.read_csv('dataset/test.csv')

In [3]:
id_ccno = txn[['id', 'old_cc_no']].drop_duplicates()
train_id_ccno = pd.merge(id_ccno, train_set, on='id')

In [4]:
txn_n_daily = txn[['id', 'old_cc_no', 'date_idx', 'n4', 'n5', 'n6', 'n7']].copy()
txn_n_daily['count'] = 1
txn_n_daily = txn_n_daily.groupby(['id', 'old_cc_no', 'date_idx']).sum()

In [5]:
scaled_txn_n_daily = txn_n_daily

In [6]:
scaled_txn_n_daily.head()

n4    n5    n6    n7  count
id old_cc_no date_idx                                
1  7051      1          6000  1236  5997  2700     12
             2         33000   615  3529  1350      6
             3          5000   904  4300  2025      9
             4          9000  1011  5000  2250     10
             5          3000   618  3000  1350      6

In [7]:

txn_scaler = StandardScaler()
scaled_txn_n_daily[['n4', 'n5', 'n6', 'n7', 'count']] = txn_scaler.fit_transform(txn_n_daily)

In [8]:
train_txn = scaled_txn_n_daily.reset_index().set_index('id').sort_values(['old_cc_no', 'date_idx']).sort_index()

In [9]:
train_txn = train_txn.loc[:100001]

In [10]:
train_txn = train_txn.sort_values(['old_cc_no', 'date_idx'])

In [11]:
def create_daily_seq_from_group(group):
    padding_value = txn_scaler.transform([[0, 0, 0, 0, 0]])
    seq = np.ones((365, 5)) * padding_value

    for index, row in group.iterrows():
        seq[int(row['date_idx']) - 1] = row[['n4', 'n5', 'n6', 'n7', 'count']].to_numpy()
    return seq
def create_daily_seq(txn):
    grouped = txn.groupby(['old_cc_no'])
    seqs = grouped.apply(create_daily_seq)
    seqs = np.asarray([seq for seq in seqs])
    return seqs

In [ ]:
seqs = create_daily_seq(train_txn)

In [13]:
len(train_txn['old_cc_no'].drop_duplicates())

150512

In [283]:
train_old_ccno = train_txn['old_cc_no'].drop_duplicates().sort_values()

In [311]:
labels = train_id_ccno.set_index('old_cc_no').loc[train_old_ccno].sort_values('old_cc_no')['label']

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


In [308]:
labels = np.array([to_categorical(label, 13) if not np.isnan(label) else to_categorical(0, 13) for label in labels])

In [310]:
labels.shape

(150512, 13)

In [315]:
train_old_ccno

301160